## Example

In [ ]:
X = create_mob(pd.read_csv('input_delinquent_application_202111.csv'))

In [ ]:
Fraud = pd.read_csv('delinquent_application_flag.csv')[["apl_no","flag_fraud"]]\
.rename(columns={"apl_no":"apl_grp_no","flag_fraud":"flag"})

In [ ]:
X = X.merge(Fraud, how='left', on="apl_grp_no").fillna(0)
del Fraud

In [ ]:
dpd_cols = ["M0","M1","M2","M3","M4","M5"]
osb_cols = ["M0_OS","M1_OS","M2_OS","M3_OS","M4_OS","M5_OS"]
colors   = ["#1B9CFC","#CAD3C8","#F97F51","#FC427B"]

In [ ]:
X["clean"] = X[dpd_cols].sum(1)==0
X["flag"]  = np.where(X["flag"]==1,"Fraud", "Normal")
cat = {"DS":"Self-Apply", "EA":"E_App", "QR":"Paper"}
X["apl_grp_type"] = X["apl_grp_type"].apply(lambda x:cat[x[:2]])
X["M0_pWDL"] = X["M0_PNP"] / X["fnl_cr_lmt"]
X["M1_pWDL"] = X["M1_PNP"] / X["fnl_cr_lmt"]

In [ ]:
X.head()

## Averge delinquency

In [ ]:
# "flag" "cust_type","apl_grp_type", "pd_lvl2"
groupby = ["pd_lvl2", "cust_type"] 
ax = plot_avgdpd(X.loc[X["clean"]==False], groupby, dpd_cols, 
                 start_mth=0, colors=colors, focus="M3", n_tops=3, 
                 ax=plt.subplots(figsize=(9.5,4.5))[1])

In [ ]:
groupby_table(X.loc[X["clean"]==False], groupby, dpd_cols)

In [ ]:
groupby_table(X.loc[X["clean"]==False], groupby, osb_cols, 
              np.nansum, factor=1/10**6)

## Vintage analysis

In [ ]:
# "flag" "cust_type","apl_grp_type", "pd_lvl2"
groupby = ["apl_grp_type", "cust_type"] 
for dpd_geq,focus in zip([30, 60, 90],["M2","M3","M4"]):
    ax = plt.subplots(figsize=(9.5,4.3))[1]
    ax = plot_vintage(X.loc[(X["clean"]==False)], groupby, dpd_cols, 
                      dpd_geq, ax=ax, colors=colors, focus=focus, n_tops=3)

In [ ]:
vintage_table(X.loc[X["clean"]==False], groupby, dpd_cols)

## Roll rate analysis

In [ ]:
mobs, cond = ["M3","M4"], (X["clean"]==False) 
ax = plot_rollrate(X.loc[cond].copy(), mobs)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8.3*2,6.5))
fig.subplots_adjust(hspace=0.3)
mobs, cond = ["M3","M4"], (X["clean"]==False)
labels = np.r_[["{} Months on book".format(s[1:]) for s in mobs]][::-1]
axes[0] = plot_rollrate(X.loc[cond].copy(), mobs, ax=axes[0], labels=labels)
axes[1] = plot_rollrate(X.loc[cond].copy(), mobs, ax=axes[1], labels=labels,
                        values=X["M4_OS"]/10**6, num_format="{:,.1f}M".format)
axes[0].set_title("Number of applications\n", fontsize=25)
axes[1].set_title("O/S amount (million)\n", fontsize=25)
plt.tight_layout()

## % Withdrawal @ M0, and M1
add 90% flag

In [ ]:
# "flag","pd_lvl2", "cust_type", "apl_grp_type"]
groupby= ["flag","pd_lvl2", "cust_type"]
aggfnc = {"ip_id" : "count", "M0_pWDL": "mean", "M1_pWDL": "mean"}
rename = {"ip_id":"N", "M0_pWDL": "M0", "M1_pWDL": "M1"}
X.loc[X["clean"]==False].groupby(groupby).agg(aggfnc).rename(columns=rename)